In [1]:
import pandas as pd
import numpy as np
# from scipy.stats import pearsonr, median_abs_deviation
# from sklearn import metrics
import sys
import os
from statsmodels.stats.multitest import multipletests
from scipy.stats import combine_pvalues, norm, mannwhitneyu, spearmanr, pearsonr
from read_clustering.variant_call import VariantCalls
import matplotlib.pyplot as plt
from scipy.stats import shapiro, combine_pvalues
import seaborn as sns
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
from scipy.stats import combine_pvalues, norm, mannwhitneyu
from scipy.stats import chi2_contingency
from EmpiricalBrownsMethod import EmpiricalBrownsMethod

/Users/andrewbailey/anaconda3/envs/read_clustering/lib/python3.7/site-packages/umap/__init__.py:9: UserWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn("Tensorflow not installed; ParametricUMAP will be unavailable")
/Users/andrewbailey/anaconda3/envs/read_clustering/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# final ivt vs native
CBF5GAL = "modification_calls/20190610_R941_CBF5GAL.csv"
CBF5GLU = "modification_calls/20190611_R941_CBF5GLU.csv"
NOP58GAL = "modification_calls/20190617_R941_NOP58GAL.csv"
NOP58GLU = "modification_calls/20190619_R941_NOP58GLU.csv"
IVT = "modification_calls/20191014_R941_IVT.csv"
mutant448 = "modification_calls/20210405_R941_mutant448.csv"
WT_YPD = "modification_calls/20210410_R941_WT_YPD.csv"
WT_KOAC = "modification_calls/20210412_R941_WT_KOAC.csv"
mutant452 = "modification_calls/20210413_R941_mutant452.csv"
WT_CY = "modification_calls/20210414_R941_WT_CY.csv"
mutant451 = "modification_calls/20210415_R941_mutant451.csv"
WT_PB = "modification_calls/20210520_R941_WT_PB.csv"
WT_stationary = "modification_calls/20210521_R941_WT_stationary.csv"
Dbp3d_YPD = "modification_calls/20210527_R941_Dbp3d_YPD.csv"
trl1d_YPD = "modification_calls/20210528_R941_trl1d_YPD.csv"
WT_rapa5 = "modification_calls/20210531_R941_WT_rapa5.csv"
Mix = "modification_calls/20210618_R941_Mix.csv"
prp43 = "modification_calls/20210619_R941_prp43.csv"
MIX_2 = "modification_calls/20210716_R941_MIX.csv"
YS602_18_YPD = "modification_calls/20210717_R941_YS602_18_YPD.csv"
prp16_18deg_YPD = "modification_calls/20210719_R941_prp16_18deg_YPD.csv"
spp382_YPD = "modification_calls/20210720_R941_spp382_YPD.csv"
YS602_18_YPD2 = "modification_calls/20210722_R941_YS602_18_YPD2.csv"
dbr1d_YPD = "modification_calls/20210729_R941_dbr1d_YPD.csv"
sqs1d_YPD = "modification_calls/20210730_R941_sqs1d_YPD.csv"
snR4d_YPD = "modification_calls/20210823_R941_snR4d_YPD.csv"
snR45d_YPD = "modification_calls/20210824_R941_snR45d_YPD.csv"
mutant448_2 = "modification_calls/20210825_R941_mutant448.csv"
snR4d_alpha_YPD = "modification_calls/20210902_R941_snR4d_alpha_YPD.csv"
WT_YPD2 = "modification_calls/20210930_R941_YS602_WT_repeat2.csv"
WT_YPD3 = "modification_calls/20211001_R941_YS602_WT_repeat3.csv"
prx1 = "modification_calls/20211006_R941_pxr1d_YPD.csv"
YS602_18_YPD3 = "modification_calls/20211007_R941_YS602_18_YPD3.csv"
prp43_2 = "modification_calls/20211008_R941_prp43_18deg_YPD_repeat2.csv"
Dbp3d_YPD2 = "modification_calls/20211011_R941_dbp3d_YPD_repeat2.csv"
prx1_2 = "modification_calls/20211021_pxr1d_YPD_repeat2.csv"


In [3]:
correlation_df = pd.read_csv("figures/summary_data/all_experiment_correlations_corrected_pvalues.csv", 
                                   index_col=0)
chi2_pvalues_df = pd.read_csv("figures/summary_data/all_experiment_percent_modification_corrected_pvalues_chi2.csv", 
                                   index_col=0)


In [89]:
def get_max_comparison_pvalues(labels, compare_labels=["WT_YPD", "WT_YPD2", "WT_YPD3"]):
    selections = []
    for sample in labels:
        for compare in compare_labels:
            selections.append(f"{sample}_{compare}_comparison_pvalue")   
    max_pvalues = np.max(correlation_df[selections], axis=1)
    max_pvalues = np.array([x if x != 0 else 0.00000000001 for x in max_pvalues])
    return max_pvalues

# def get_comparison_pvalues(sample, compare):
#     pvalues = correlation_df[f"{sample}_{compare}_comparison_pvalue"]
#     pvalues = np.array([x if x != 0 else 0.00000000001 for x in pvalues])
#     return pvalues

def get_max_brown_pvalue(experiments, comparisons, mask):
    brown_pvalues = []
    for experiment in experiments:
        for comparison in comparisons:
            input_data = correlation_df[[f"{experiment}_corr", f"{comparison}_corr"]][mask]
            pvalues = get_max_comparison_pvalues([experiment], compare_labels=comparisons)[mask]
            brown_pvalue = EmpiricalBrownsMethod(input_data.to_numpy(), pvalues, extra_info = True)
#             print(brown_pvalue)
            brown_pvalues.append(brown_pvalue[0])
    return np.max(brown_pvalues)


# P-value calculations

In [90]:
pseduo_u_pos = [775, 959, 965, 985, 989, 1003, 1041, 1051, 1055, 1109, 1123,
                     2128, 2132, 2190, 2257, 2259, 2263, 2265, 2313, 2339, 2348, 2350,
                     2415, 2734, 2825, 2864, 2879, 2922, 2943, 2974, 105, 119, 210, 301, 465, 631, 758, 765,
                     998, 1180, 1186,
                     1289, 1414]
pseduo_u_pos = [x+1 for x in pseduo_u_pos]
twoprimeo_pos = [648, 649, 662, 804, 806, 816, 866, 875, 897, 907, 1132,
                      1436, 1448, 1449, 1887, 2196, 2219, 2255, 2279, 2280, 2287, 2336,
                      2346, 2416, 2420, 2618, 2639, 2723, 2728, 2790, 2792, 2814, 2920,
                      2921, 2945, 2947, 2958, 27, 99, 413, 419, 435, 540, 561, 577, 618, 795, 973,
                      1006, 1125, 1268, 1270, 1427, 1571, 1638]
twoprimeo_pos = [x+1 for x in twoprimeo_pos]


## Depletions

# Brown's test p-value
https://academic.oup.com/bioinformatics/article/32/17/i430/2450768

In [91]:
mask = (correlation_df["ref_index1"].isin(pseduo_u_pos) & correlation_df["ref_index2"].isin(pseduo_u_pos))
comparisons = ["WT_YPD", "WT_YPD2", "WT_YPD3"]
experiments = ["CBF5GLU"]
print(get_max_brown_pvalue(experiments, comparisons, mask))
mask = (~correlation_df["ref_index1"].isin(pseduo_u_pos) & ~correlation_df["ref_index2"].isin(pseduo_u_pos))
print(get_max_brown_pvalue(experiments, comparisons, mask))


5.492161658647435e-05
1.0


In [92]:
mask = (correlation_df["ref_index1"].isin(twoprimeo_pos) & correlation_df["ref_index2"].isin(twoprimeo_pos))
comparisons = ["WT_YPD", "WT_YPD2", "WT_YPD3"]
experiments = ["NOP58GLU"]
print(get_max_brown_pvalue(experiments, comparisons, mask))
mask = (~correlation_df["ref_index1"].isin(twoprimeo_pos) & ~correlation_df["ref_index2"].isin(twoprimeo_pos))
print(get_max_brown_pvalue(experiments, comparisons, mask))


1.4970039431451155e-16
0.999999999759188


In [93]:
experiments = ["NOP58GLU"]
mask = (correlation_df["ref_index1"].isin([1280, 1773]) & correlation_df["ref_index2"].isin(twoprimeo_pos))
comparisons = ["WT_YPD", "WT_YPD2", "WT_YPD3"]
print(get_max_brown_pvalue(experiments, comparisons, mask))

2.2502355315796776e-05


In [94]:
experiments = ["CBF5GLU"]
mask = (correlation_df["ref_index1"].isin([1191]) & correlation_df["ref_index2"].isin(pseduo_u_pos))
comparisons = ["WT_YPD", "WT_YPD2", "WT_YPD3"]
print(get_max_brown_pvalue(experiments, comparisons, mask))

5.389532176727951e-07


# Mixture and snoRNA KO's

In [99]:
experiments = ["Mix", "MIX_2"]
positions = [436, 759, 1290, 1415]

mask = (correlation_df["ref_index1"].isin(positions) & correlation_df["ref_index2"].isin(positions))  
one_mask = ~((correlation_df["ref_index1"] == 1290) & (correlation_df["ref_index2"] == 1415))
mask = mask & one_mask
comparisons = ["WT_YPD", "WT_YPD2", "WT_YPD3"]
print(get_max_brown_pvalue(experiments, comparisons, mask))
print(get_max_comparison_pvalues(experiments)[~one_mask])

3.509419606371858e-08
[6.36098465e-16]


In [96]:
chi2_pvalues_df.loc[[436, 759, 1290, 1415]][["Mix_corrected_pvalue", "MIX_2_corrected_pvalue"]].max(axis=1)

reference_index
436     6.037790e-15
759     2.313304e-34
1290    2.313304e-34
1415    3.961028e-05
dtype: float64

In [97]:
MIX = chi2_pvalues_df["MIX_2_mean"][[436, 759, 1290, 1415]]
WT = chi2_pvalues_df["WT_YPD_mean"][[436, 759, 1290, 1415]]
WT2 = chi2_pvalues_df["WT_YPD2_mean"][[436, 759, 1290, 1415]]
WT3 = chi2_pvalues_df["WT_YPD3_mean"][[436, 759, 1290, 1415]]

In [98]:
pd.DataFrame([(MIX - WT), (MIX - WT2), (MIX - WT3)])

reference_index,436,759,1290,1415
0,-0.163803,-0.228544,-0.185225,-0.141084
1,-0.176463,-0.230526,-0.202300,-0.134405
2,-0.171986,-0.243004,-0.199668,-0.161221


## Wild Type

In [100]:
experiment = "WT_YPD"
pos = [632, 766, 100 ,436, 1773, 1639]
max_pvalues = get_max_comparison_pvalues(["WT_YPD", "WT_YPD2", "WT_YPD3"], compare_labels=["IVT"])

for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, max_pvalues[index])


100 436 0.0003173944331567858
100 632 0.9445677763941264
100 766 0.13941750432532907
100 1639 0.3287868129604332
100 1773 0.3523166141703712
436 632 0.04141432857375133
436 766 0.3514259925240414
436 1639 0.2610225602126527
436 1773 0.007950229580684136
632 766 0.00013240582931339312
632 1639 0.7916627015210821
632 1773 0.1165595492621404
766 1639 0.05725470177481661
766 1773 0.9054841799047364
1639 1773 4.547813599123047e-06


In [101]:
experiments = ["WT_YPD", "WT_YPD2", "WT_YPD3"]
positions = [1437, 1449, 1450]
mask = (correlation_df["ref_index1"].isin(positions) & correlation_df["ref_index2"].isin(positions))  
comparisons = ["IVT"]
print(get_max_brown_pvalue(experiments, comparisons, mask))

1.0454393166593538e-44
(9.651121925119254e-43, 6.434476248597971e-62, 1.4804530139182013, 4.052813526395046, 300.4865624048362)


## RNA helicases

In [102]:
experiments = ["Dbp3d_YPD", "Dbp3d_YPD2"]
positions = [1437, 1449, 1450]
mask = (correlation_df["ref_index1"].isin(positions) & correlation_df["ref_index2"].isin(positions))  
comparisons = ["WT_YPD", "WT_YPD2", "WT_YPD3"]
print(get_max_brown_pvalue(experiments, comparisons, mask))


max_pvalues = get_max_comparison_pvalues(experiments)
pos = [817, 908, 2619, 2724]
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, max_pvalues[index])




3.5794266188591064e-68
817 908 2.3711432447091034e-33
817 2619 0.3181658027751019
817 2724 0.19031140679634995
908 2619 0.8378461466963627
908 2724 0.5239310399298436
2619 2724 3.4825501712230022e-22


In [103]:
experiments = ["prp43", "prp43_2"]
positions = [1437, 1449, 1450]
mask = (correlation_df["ref_index1"].isin(positions) & correlation_df["ref_index2"].isin(positions))  
comparisons = ["WT_YPD", "WT_YPD2", "WT_YPD3"]
print(get_max_brown_pvalue(experiments, comparisons, mask))


max_pvalues = get_max_comparison_pvalues(experiments)
pos = [817, 908, 2619, 2724]
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, max_pvalues[index])


3.5074823309232335e-11
817 908 0.34584992246726376
817 2619 0.8606628604773594
817 2724 0.7854867938574578
908 2619 0.8157770040743397
908 2724 0.675727100323513
2619 2724 0.9190410711366266


In [104]:
experiments = ["prx1", "prx1_2"]
positions = [1437, 1449, 1450]
mask = (correlation_df["ref_index1"].isin(positions) & correlation_df["ref_index2"].isin(positions))  
comparisons = ["WT_YPD", "WT_YPD2", "WT_YPD3"]
print(get_max_brown_pvalue(experiments, comparisons, mask))


max_pvalues = get_max_comparison_pvalues(experiments)
pos = [817, 908, 2619, 2724]
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, max_pvalues[index])


1.8473122575755324e-55
817 908 4.713263523264463e-17
817 2619 0.19774066317101055
817 2724 0.9330873895521664
908 2619 0.9998619669966644
908 2724 0.8373976768693838
2619 2724 0.0007229056736246494
